# Demand Forecasting – Data Cleaning (v1)

## Objective
The goal of this notebook is to clean and normalize raw pharmacy sales data
to prepare it for demand forecasting.

This step focuses on:
- Isolating true sales transactions
- Standardizing product names
- Ensuring quantity and time consistency

⚠️ This notebook does NOT include:
- Aggregation
- Feature engineering
- Modeling


## Dataset Description

The dataset contains transaction-level sales data exported from a pharmacy POS system.
Each row represents a single transaction event.

Key characteristics:
- Arabic and English product names
- Noisy text (e.g. "سعر جديد", "#")
- Multiple operational columns not required for forecasting


In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
df = pd.read_excel("../data/raw/sales.xlsx")

In [ ]:
df.head(10)

In [ ]:
df.info()


In [ ]:
df.shape


## Column Relevance Analysis

Based on the initial inspection, the dataset contains several operational
and metadata columns that are not directly relevant to demand forecasting.

For version v1 of this project, the focus is strictly on demand signals.
Therefore, columns are evaluated as follows:

### Columns to Keep
- `اسم الصنف`: Core identifier for the product.
- `التاريخ`: Required to build the time dimension.
- `الوقت`: Used to construct a full datetime field.
- `العبوات` and `الأجزاء`: Required to infer actual demand quantity.
- `العملية`: Used to filter true sales transactions.

### Columns to Exclude
- Price-related columns (`سعر البيع`, `الأجمالى`)
- Stock balance columns (`الرصيد الكلى الجديد`, `الرصيد الجديد للمخزن`)
- Expiry date (`تاريخ الصلاحية`)
- User, machine, and customer/supplier information
- Excel artifact column (`Unnamed: 16`)

These excluded columns are considered noise for demand forecasting v1
and may be revisited in future iterations.


In [ ]:
columns_needed = [
    "اسم الصنف",
    "التاريخ",
    "الوقت",
    "العبوات",
    "الأجزاء",
    "العملية"
]

df = df[columns_needed].copy()


In [ ]:
df.head()


In [ ]:
cases = {
    "packs_negative": (df["العبوات"] < 0).sum(),
    "parts_negative": (df["الأجزاء"] < 0).sum(),
    "both_negative": ((df["العبوات"] < 0) & (df["الأجزاء"] < 0)).sum()
}

cases

## Quantity Representation Analysis

An analysis of quantity-related columns revealed that sales transactions
are not consistently represented using a single field.

### Observed Patterns
- In many transactions, demand is represented as a negative value in the `العبوات` column.
- In other transactions, demand is represented as a negative value in the `الأجزاء` column.
- A very small number of rows contain negative values in both columns simultaneously.

Summary statistics:
- Rows with negative `العبوات`: 50,763
- Rows with negative `الأجزاء`: 41,109
- Rows with both values negative: 82 (edge cases)

This confirms that demand signals are split across two columns depending on
how the POS system records the transaction.

---

## Unified Demand Signal Decision

To preserve demand information without introducing unreliable assumptions
(e.g. part-to-pack conversions), a unified demand representation will be used.

A new column `qty_sold` will be created with the following rules:
- If `العبوات < 0`, then `qty_sold = abs(العبوات)`
- Else if `الأجزاء < 0`, then `qty_sold = 1`
- Rows where neither condition applies will be excluded
- Rows where both values are negative are treated as a single demand event

This approach captures true demand occurrence while remaining robust
for demand forecasting in version v1 of the project.

In [ ]:
df["qty_sold"] = 0


In [ ]:
mask_packs = df["العبوات"] < 0
df.loc[mask_packs, "qty_sold"] = df.loc[mask_packs, "العبوات"].abs()

In [ ]:
mask_parts = (df["qty_sold"] == 0) & (df["الأجزاء"] < 0)
df.loc[mask_parts, "qty_sold"] = 1

In [ ]:
df = df[df["qty_sold"] > 0].copy()

In [ ]:
df["qty_sold"].describe()


In [ ]:
df[["العبوات", "الأجزاء", "qty_sold"]].head(10)

## Unified Demand Column Validation

The newly created `qty_sold` column was validated to ensure it correctly
represents demand signals across all transactions.

### Summary Statistics
- Total rows after filtering: 91,790
- Median demand per transaction: 1 unit
- Mean demand per transaction: ~1.09 units
- Minimum value: 1
- Maximum value: 100

This distribution indicates that most sales transactions represent
single-unit demand, which is expected in a retail pharmacy setting,
with occasional bulk sales.

### Sample Validation
Inspection of individual rows confirms that:
- Transactions recorded via negative `الأجزاء` values are correctly mapped to `qty_sold = 1`
- Transactions recorded via negative `العبوات` values are mapped to `qty_sold = abs(العبوات)`
- No zero or negative values remain in the unified demand column

These results confirm that `qty_sold` provides a clean and consistent
representation of demand suitable for downstream aggregation and forecasting.

---

## Datetime Construction

To enable time-based analysis and forecasting, a unified datetime column
will be created by combining the date and time fields.

This step ensures:
- Each transaction has a single temporal reference
- Proper chronological ordering
- Compatibility with time series aggregation

Rows with invalid or missing datetime values will be excluded.

In [ ]:
df["datetime"] = pd.to_datetime(
    df["التاريخ"].astype(str) + " " + df["الوقت"].astype(str),
    errors="coerce"
)

In [ ]:
df = df[df["datetime"].notna()].copy()


In [ ]:
df["datetime"].head()


In [ ]:
df["datetime"].dtype


In [ ]:
df = df.sort_values("datetime")


## Datetime Validation

- Date and time fields were successfully combined into a single `datetime` column.
- Rows with invalid datetime values were excluded.
- The dataset is now chronologically ordered and ready for time-based aggregation.

---

## Product Name Cleaning & Normalization

Product names contain noisy tokens and inconsistent formatting
(e.g. marketing words, symbols, mixed casing).

This step applies rule-based text normalization to:
- Remove non-identifying tokens (e.g. "#", "سعر جديد")
- Standardize casing and spacing
- Preserve product identity without translation or assumptions

The goal is to create a stable canonical product name suitable
for aggregation and forecasting.

In [ ]:
df["product_name"] = df["اسم الصنف"].astype(str).str.lower()

In [ ]:
noise_tokens = [
    "#",
    "سعر جديد",
    "new price",
    "عرض",
    "خصم"
]

for token in noise_tokens:
    df["product_name"] = df["product_name"].str.replace(token, "", regex=False)

In [ ]:
df["product_name"] = (
    df["product_name"]
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

In [ ]:
df[["اسم الصنف", "product_name"]].head(20)

In [ ]:
df["product_name"].nunique()

In [ ]:
product_counts = (
    df.groupby("product_name")
      .size()
      .reset_index(name="rows_count")
      .sort_values("rows_count", ascending=False)
)

product_counts.head(20)

In [ ]:
product_counts[product_counts["rows_count"] <= 3].head(30)


In [ ]:
from difflib import SequenceMatcher

def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()


In [ ]:
names = product_counts["product_name"].tolist()

similar_pairs = []
for i in range(len(names)):
    for j in range(i+1, min(i+200, len(names))):
        score = similarity(names[i], names[j])
        if score > 0.92 and names[i] != names[j]:
            similar_pairs.append((names[i], names[j], score))

similar_pairs[:10]


## Product Name Similarity Analysis

A textual similarity analysis was performed to identify potential duplicate
product names resulting from inconsistent formatting or spelling.

### Observations
- Several product name pairs exhibit high string similarity scores (>0.92).
- Upon inspection, these pairs represent legitimate product variants
  (e.g. different dosages, pack sizes, or formulations).
- Examples include:
  - Same brand with different strengths (e.g. 2mg vs 4mg)
  - Same product with different pack counts (e.g. 24 tablets vs 48 tablets)

### Decision
No additional merging or normalization is applied at this stage.

High textual similarity alone is not sufficient to infer duplicate products.
Preserving dosage and pack-size distinctions is critical for accurate
demand forecasting.

This confirms that the current `product_name` representation is appropriate
for version v1 of the project.

---

## Final Column Selection

After completing the data cleaning and validation steps, the dataset
contains several intermediate and raw columns that are no longer required.

For the forecasting pipeline, only columns that directly represent
*demand over time per product* are retained.

### Selected Columns
- `product_name`  
  Canonical identifier for each product after normalization and validation.
  This column is used as the primary grouping key.

- `datetime`  
  Unified timestamp representing when each demand event occurred.
  Required for time-based aggregation and forecasting.

- `qty_sold`  
  Unified demand quantity representing a single demand signal per transaction,
  constructed from raw pack/part representations without unsafe assumptions.

### Excluded Columns
All other columns (raw quantities, operation type, original date/time fields,
and operational metadata) are excluded because:
- They have already contributed to derived features
- They do not add additional information for demand forecasting
- Retaining them increases noise and complexity without analytical benefit

This final column selection represents the clean contract between
the data cleaning phase and downstream aggregation and modeling steps.


In [ ]:
final_df = df[["product_name", "datetime", "qty_sold"]].copy()

In [ ]:
os.makedirs("../data/processed", exist_ok=True)
final_df.to_csv(
    "../data/processed/clean_sales.csv",
    index=False
)

In [ ]:
# Data Quality

raw_rows = 93861
clean_rows = final_df.shape[0]

metrics = {
    "raw_rows": raw_rows,
    "clean_rows": clean_rows,
    "rows_removed": raw_rows - clean_rows,
    "rows_removed_pct": round((raw_rows - clean_rows) / raw_rows * 100, 2),
    "unique_products": final_df["product_name"].nunique(),
    "median_qty_sold": final_df["qty_sold"].median(),
    "max_qty_sold": final_df["qty_sold"].max()
}

metrics


## Data Quality Metrics

The following metrics summarize the impact of the data cleaning process
on the original dataset.

### Dataset Size
- Raw rows: 93,861
- Rows after cleaning: 91,790
- Rows removed: 2,071 (~2.2%)

Removed rows primarily correspond to:
- Non-sales operations
- Transactions without valid demand signals
- Rows with invalid datetime values

### Demand Characteristics
- Number of unique products: 4,675
- Median demand per transaction: 1 unit
- Maximum observed demand in a single transaction: 100 units

These metrics indicate that the cleaning process preserved the vast majority
of demand information while removing noisy and invalid records.

---

## Data Contract: `clean_sales.csv`

The output of this notebook is a clean, validated dataset that serves
as the sole input for downstream aggregation and forecasting steps.

### Schema
| Column Name   | Type        | Description |
|---------------|-------------|-------------|
| product_name  | string      | Canonical product identifier after normalization |
| datetime      | datetime64  | Timestamp of the demand event |
| qty_sold      | integer     | Unified demand quantity per transaction |

### Guarantees
- No missing values in any column
- `qty_sold` contains only positive integers
- Each row represents a single valid demand event
- Dataset is chronologically sortable by `datetime`

Any downstream notebook or pipeline stage must rely exclusively on this schema.

In [39]:
# Time Coverage Check

time_coverage = {
    "start_date": final_df["datetime"].min(),
    "end_date": final_df["datetime"].max(),
    "total_days": (final_df["datetime"].max() - final_df["datetime"].min()).days
}

time_coverage


{'start_date': Timestamp('2025-02-07 14:40:00'),
 'end_date': Timestamp('2026-01-11 23:33:00'),
 'total_days': 338}

## Time Coverage Validation

The cleaned dataset spans a sufficiently long and continuous historical period
to support demand forecasting.

- Start date: February 7, 2025
- End date: January 11, 2026
- Total time coverage: 338 days (~11 months)

This time span provides adequate temporal depth for weekly demand aggregation
and forecasting, allowing models to learn short-term patterns and medium-term
demand trends.

While a full multi-year seasonal cycle is not available, the current coverage
is appropriate for version v1 of the forecasting pipeline.

Future iterations may incorporate longer historical windows to improve
seasonality modeling once additional data becomes available.

---

## Assumptions & Limitations

- Demand is represented at transaction level, not customer level
- Pack-to-part conversion is intentionally ignored in v1
- No promotion, seasonality, or supplier constraints are considered
- Low-frequency products will be handled separately during aggregation.

---


## Cleaning Summary (v1)

This notebook completed the data cleaning phase for the pharmacy
demand forecasting project.

Key outcomes:
- Unified demand representation via `qty_sold`
- Canonical product naming without unsafe merging
- Validated datetime construction
- Removal of operational and non-relevant columns

The resulting dataset is saved as `clean_sales.csv` and will be used
as the sole input for aggregation and forecasting steps.

## Why This Cleaning Matters for Forecasting

The decisions made in this notebook ensure that downstream forecasting models
learn true demand patterns rather than artifacts of POS system representation.

By unifying demand signals, preserving product identity, and enforcing a
strict data contract, the aggregation and modeling stages can focus on
temporal demand dynamics with minimal noise and leakage.